# Get facial landmarks coordinates with dlib

In [1]:
import dlib

def get_landmarks(filename, predictor_path = "data/shape_predictor_68_face_landmarks.dat"):
    
    # Instantiate a face detector
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    
    img = dlib.load_rgb_image(filename)

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    detected = detector(img, 1) # Return corresponding rectangles for the detected faces
    for index, box in enumerate(detected):
        landmarks = predictor(img, box)  
        coords = []
        for i in range(0, 68):
            coords.append((landmarks.part(i).x, landmarks.part(i).y))

    return coords

# Face Swap

In [2]:
import sys
import numpy as np
import cv2

# Apply affine transform calculated using srcTri and dstTri to src and output an image of certain size
def apply_affine_transform(src, srcTri, dstTri, size):
    
    # Given a pair of triangles, find the affine transform needed
    warp_mat = cv2.getAffineTransform(np.float32(srcTri), np.float32(dstTri))
    
    # Apply the above Affine transform to the src image
    dst = cv2.warpAffine(src, warp_mat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    
    return dst


# Check if a point is inside of a rectangle, this is a must for `Subdiv2D`
def is_inside(rect, point):
    if point[0] < rect[0]:
        return False
    elif point[1] < rect[1]:
        return False
    elif point[0] > rect[0] + rect[2]:
        return False
    elif point[1] > rect[1] + rect[3]:
        return False
    return True


# Calculate delaunay triangles
def calculate_delaunay_triangles(rect, points):
    # Create a Subdiv2D object
    subdiv = cv2.Subdiv2D(rect)
    
    # Insert points into subdiv
    for p in points:
        subdiv.insert(p)
    
    # Return a list of all possible triangles
    # Each triangle is a 6 number vector
    triangles = subdiv.getTriangleList()
    
    delaunay_triangles = []
    
    pts = [] # To store the indices corresponding with triangles generaged
    
    for t in triangles:
        pts.append((t[0], t[1]))
        pts.append((t[2], t[3]))
        pts.append((t[4], t[5]))
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        
        if is_inside(rect, pt1) and is_inside(rect, pt2) and is_inside(rect, pt3):
            indices = []
            # Get face points by coordinates
            for j in range(0, 3):
                for k in range(0, len(points)):
                    if abs(pts[j][0] - points[k][0]) < 1.0 and abs(pts[j][1] - points[k][1]) < 1.0:
                        indices.append(k)
            
            # Make sure there are three points to form a triangle
            if len(indices) == 3:
                delaunay_triangles.append((indices[0], indices[1], indices[2]))
                
        pts = []
        
    return delaunay_triangles


# Warps triangular regions from img1 and img2
def warp_triangle(img1, img2, t1, t2):
    
    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    
    # Offset points by left top corner of the respective rectangles
    t1_rect = []
    t2_rect = []
    t2_rect_int = []
    
    for i in range(0, 3):
        t1_rect.append(((t1[i][0] - r1[0]), (t1[i][1] - r1[1])))
        t2_rect.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))
        t2_rect_int.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))
        
    # Get a mask by filling the triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(t2_rect_int), (1.0, 1.0, 1.0), 16, 0)
    
    # Apply warp to small rectangular patches
    img1_rect = img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]]
    
    size = (r2[2], r2[3])
    
    img2_rect = apply_affine_transform(img1_rect, t1_rect, t2_rect, size)
    
    img2_rect = img2_rect * mask
    
    # Copy triangular region of the rectangular path to the output image
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ((1.0, 1.0, 1.0) - mask)
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2_rect


In [3]:
filename1 = "images/ted_cruz.jpg"
filename2 = "images/donald_trump.jpg"

img1 = cv2.imread(filename1)
img2 = cv2.imread(filename2)
# image 2 the destinate image for img1 
img1_warped = np.copy(img2)

# Get landmarks
points1 = get_landmarks(filename1)
points2 = get_landmarks(filename2)

# find corresponding convex hull points
hull1 = []
hull2 = []

# Get the indices corresponding to the convex hull points
# The returnPoints was set true because we need the index to get points for hull1
hull_index = cv2.convexHull(np.array(points2), returnPoints=False)

for i in range(0, len(hull_index)):
    hull1.append(points1[int(hull_index[i])])
    hull2.append(points2[int(hull_index[i])])
    
# Conduct Delaunay triangulation for all convex hull points
rect = (0, 0, img2.shape[1], img2.shape[0])
triangles = calculate_delaunay_triangles(rect, hull2)
#print(triangles)

if len(triangles) == 0:
    quit()
    
# Apply affine transformation to Delaunay triangles
for i in range(0, len(triangles)):
    t1 = []
    t2 = []
    
    # get triangle vertices for img1, img2
    for j in range(0, 3):
        t1.append(hull1[triangles[i][j]])
        t2.append(hull2[triangles[i][j]])
        
    warp_triangle(img1, img1_warped, t1, t2)
    
    # Debuging code to see how the facial replacement process was done
    #cv2.imshow('warped', img1_warped) 
    #cv2.waitKey(0)

# Set up a mask
hull_masks = hull2[:]

mask = np.zeros(img2.shape, dtype=img2.dtype)

cv2.fillConvexPoly(mask, np.int32(hull_masks), (255, 255, 255))
#cv2.imshow('mask', mask)

r = cv2.boundingRect(np.float32([hull2]))

center = ((r[0]+int(r[2]/2), r[1]+int(r[3]/2)))

# Clone seamlessly
output = cv2.seamlessClone(np.uint8(img1_warped), img2, mask, center, cv2.NORMAL_CLONE)

cv2.imshow("Face swapped", output)
cv2.waitKey(0)

cv2.destroyAllWindows()

<br>
<figure>
  <img src = "images/presidential-candidates.jpg" width = "100%" style = "border: thin silver solid; padding: 1px">
      <figcaption style = "text-align:left; font-style:italic">Results of the face swap</figcaption>
</figure> 
<br>

# Demonstration for Delaunay Triangle

In [6]:
import cv2
import numpy as np
import random

# Check if a point is inside a rectangle
def rect_contains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True

# Draw a point
def draw_point(img, p, color):
    cv2.circle(img, p, 2, color, cv2.FILLED, cv2.LINE_AA, 0)


# Draw delaunay triangles
def draw_delaunay(img, subdiv, delaunay_color):

    triangleList = subdiv.getTriangleList()
    size = img.shape
    r = (0, 0, size[1], size[0])

    for t in triangleList :
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        
        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3):
        
            cv2.line(img, pt1, pt2, delaunay_color, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt2, pt3, delaunay_color, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt3, pt1, delaunay_color, 1, cv2.LINE_AA, 0)


# Draw voronoi diagram
def draw_voronoi(img, subdiv):

    (facets, centers) = subdiv.getVoronoiFacetList([])

    for i in range(0,len(facets)):
        ifacet_arr = []
        for f in facets[i] :
            ifacet_arr.append(f)
        
        ifacet = np.array(ifacet_arr, np.int)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        cv2.fillConvexPoly(img, ifacet, color, cv2.LINE_AA, 0)
        ifacets = np.array([ifacet])
        cv2.polylines(img, ifacets, True, (0, 0, 0), 1, cv2.LINE_AA, 0)
        cv2.circle(img, (centers[i][0], centers[i][1]), 3, (0, 0, 0), cv2.FILLED, cv2.LINE_AA, 0)


if __name__ == '__main__':

    # Define window names
    win_delaunay = "Delaunay Triangulation"
    win_voronoi = "Voronoi Diagram"

    # Turn on animation while drawing triangles
    animate = True
    
    # Define colors for drawing.
    delaunay_color = (255,255,255)
    points_color = (0, 0, 255)

    # Read in the image.
    img = cv2.imread("images/hillary_clinton.jpg")
    
    # Keep a copy around
    img_orig = img.copy()
    
    # Rectangle to be used with Subdiv2D
    size = img.shape
    rect = (0, 0, size[1], size[0])
    
    # Create an instance of Subdiv2D
    subdiv = cv2.Subdiv2D(rect)

    # Create an array of points.
    points = get_landmarks("images/hillary_clinton.jpg")
    
    
    # Insert points into subdiv
    for p in points:
        subdiv.insert(p)
        
        # Show animation
        if animate :
            img_copy = img_orig.copy()
            # Draw delaunay triangles
            draw_delaunay(img_copy, subdiv, (255, 255, 255))
            cv2.imshow(win_delaunay, img_copy)
            cv2.waitKey(100)

    # Draw delaunay triangles
    draw_delaunay(img, subdiv, (255, 255, 255))

    # Draw points
    for p in points:
        draw_point(img, p, (0,0,255))

    # Allocate space for voronoi Diagram
    img_voronoi = np.zeros(img.shape, dtype = img.dtype)

    # Draw voronoi diagram
    draw_voronoi(img_voronoi,subdiv)

    # Show results
    cv2.imshow(win_delaunay,img)
    cv2.imshow(win_voronoi,img_voronoi)
    cv2.waitKey(0)

<br>
<figure>
  <img src = "images/delaunay_triangulation.jpg" width = "100%" style = "border: thin silver solid; padding: 1px">
      <figcaption style = "text-align:left; font-style:italic">Demonstration for the Delaunay triangulation</figcaption>
</figure> 
<br>

# Apply affine transform to a triangle of an image

In [41]:
# Copyright (c) 2016 Satya Mallick <spmallick@learnopencv.com>
# All rights reserved. No warranty, explicit or implicit, provided.

import cv2
import numpy as np


# Warps and alpha blends triangular regions from img1 and img2 to img
def warpTriangle(img1, img2, tri1, tri2):
    
    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(tri1)
    r2 = cv2.boundingRect(tri2)
    
    # Offset points by left top corner of the respective rectangles
    tri1Cropped = []
    tri2Cropped = []
    
    for i in range(0, 3):
        tri1Cropped.append(((tri1[0][i][0] - r1[0]),(tri1[0][i][1] - r1[1])))
        tri2Cropped.append(((tri2[0][i][0] - r2[0]),(tri2[0][i][1] - r2[1])))

    # Crop input image
    img1Cropped = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]

    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform(np.float32(tri1Cropped), np.float32(tri2Cropped))
    
    # Apply the Affine Transform just found to the src image
    img2Cropped = cv2.warpAffine( img1Cropped, warpMat, (r2[2], r2[3]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    # Get mask by filling triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tri2Cropped), (1.0, 1.0, 1.0), 16, 0)

    img2Cropped = img2Cropped * mask
    
    # Copy triangular region of the rectangular patch to the output image
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ( (1.0, 1.0, 1.0) - mask )
    
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2Cropped


if __name__ == '__main__' :

    # Read input image
    imgIn = cv2.imread("robot.jpg")
    
    # Output image is set to white
    imgOut = 255 * np.ones(imgIn.shape, dtype = imgIn.dtype)
    
    # Input triangle
    triIn = np.float32([[[360,200], [60,250], [450,400]]])
    
    # Output triangle
    triOut = np.float32([[[400,200], [160,270], [400,400]]])
    
    # Warp all pixels inside input triangle to output triangle
    warpTriangle(imgIn, imgOut, triIn, triOut)

    # Draw triangle using this color
    color = (255, 150, 0)

    # Draw triangles in input and output images.
    cv2.polylines(imgIn, triIn.astype(int), True, color, 2, 16)
    cv2.polylines(imgOut, triOut.astype(int), True, color, 2, 16)

    cv2.imshow("Input", imgIn)
    cv2.imshow("Output", imgOut)
    
    
    cv2.waitKey(0)

<br>
<figure>
  <img src = "images/warp-triangle-opencv.jpg" width = "100%" style = "border: thin silver solid; padding: 1px">
      <figcaption style = "text-align:left; font-style:italic">Triangle Warp Demonstration</figcaption>
</figure> 
<br>